In [1]:
from mpl_toolkits.mplot3d import Axes3D
from matplotlib import pyplot, cm
import numpy as np
%matplotlib inline
from numpy import arctan, log, pi, sqrt

In [2]:
nx  = 100
ny  = 100
nt  = 100

xmin = 0
xmax = 10
ymin = 0
ymax = 10

dx = (xmax - xmin) / (nx - 1)
dy = (ymax - ymin) / (ny - 1)
dt = 0.01

x  = np.linspace(xmin, xmax, nx)
y  = np.linspace(xmin, xmax, ny)

X, Y = np.meshgrid(x, y)

In [3]:
u  = np.zeros((nx, ny))
un = np.zeros((nx, ny))

v  = np.zeros((nx, ny))
vn = np.zeros((nx, ny))

T  = np.zeros((nx, ny))
Tn = np.zeros((nx, ny))

C  = np.zeros((nx, ny))
Cn = np.zeros((nx, ny))

In [4]:
#Problem specs
a = 1
beta = 21e5
nu = 1.0546875e-06
gamma = 1
n1 = 0.6  #power on the paranthesis "n"
rho = 1024
mu0 = 4e-7*pi
M = 8.6e5
h = 12.5e-6
w = 25e-6
c = 4179
kinf = 0.613
delT = 100
eps = 0.2
Tinf = 373.15
tau = 1
Q0 = 50
DB = 1
DT = 1

Uw = 1
Vw = 1
hf = 1
Tf = 1
hm = 1
Cf = 1
Cinf = 100

In [5]:
def Hcalc(M, i, j, h, w):
    H = M**2*(-4*h*((4*h**2*(w - i)**2 + (-h**2 + j**2 + (w - i)**2)**2)*(-h**2 + j**2 - (w + i)**2) + (4*h**2*(w + i)**2 + (-h**2 + j**2 + (w + i)**2)**2)*(h**2 - j**2 + (w - i)**2))*((-h + j)**2 + (-w + i)**2)*((-h + j)**2 + (w + i)**2)*((h + j)**2 + (-w + i)**2)*((h + j)**2 + (w + i)**2)*(arctan(2*h*(-w + i)/(-h**2 + j**2 + (-w + i)**2)) - arctan(2*h*(w + i)/(-h**2 + j**2 + (w + i)**2))) + (4*h**2*(w - i)**2 + (-h**2 + j**2 + (w - i)**2)**2)*(4*h**2*(w + i)**2 + (-h**2 + j**2 + (w + i)**2)**2)*(-(w + i)*(-(-h + j)**2 + (h + j)**2)*((-h + j)**2 + (-w + i)**2)*((h + j)**2 + (-w + i)**2) + ((-w + i)*((h + j)**2 + (-w + i)**2) + (w - i)*((-h + j)**2 + (-w + i)**2))*((-h + j)**2 + (w + i)**2)*((h + j)**2 + (w + i)**2))*(log(((-h + j)**2 + (-w + i)**2)/((h + j)**2 + (-w + i)**2)) - log(((-h + j)**2 + (w + i)**2)/((h + j)**2 + (w + i)**2))))/(2*pi*sqrt(M**2*((log(((h - j)**2 + (w - i)**2)/((h + j)**2 + (w - i)**2)) - log(((h - j)**2 + (w + i)**2)/((h + j)**2 + (w + i)**2)))**2 + 4*(arctan(2*h*(w - i)/(-h**2 + j**2 + (w - i)**2)) + arctan(2*h*(w + i)/(-h**2 + j**2 + (w + i)**2)))**2))*(4*h**2*(w - i)**2 + (-h**2 + j**2 + (w - i)**2)**2)*(4*h**2*(w + i)**2 + (-h**2 + j**2 + (w + i)**2)**2)*((-h + j)**2 + (-w + i)**2)*((-h + j)**2 + (w + i)**2)*((h + j)**2 + (-w + i)**2)*((h + j)**2 + (w + i)**2))
    return H

In [6]:
u[:, 0] = Uw
v[:, 0] = Vw

T[:, 1] = -((hf*dy)/kinf) * (Tf - T[:, 0]) + T[:, 0]
C[:, 1] = -((hm*dy)/DB) * (Cf - C[:, 0]) + C[:, 0]
T[:, ny-1] = Tinf
C[:, ny-1] = Cinf
v[:, ny-1] = 0

In [ ]:
#Boundary conditions:

for n in range(nt):
    for i in range(nx):
        for j in range(ny):
            u[i, ny-1] = ((a*i)/(1-beta*n))

In [ ]:
udiff = 1
Tdiff = 1
Cdiff = 1
stepcount = 0
accu = 0.001

while (udiff > accu) and (Tdiff > accu) and (Cdiff > accu):
    un = u.copy()
    vn = v.copy()
    Tn = T.copy()
    Cn = C.copy()
    
    for n in range(nt):
        for i in range(nx-2):
            for j in range(1, ny-2):
                #Velocity
                u[i, j] = dt * (((a*i*(a+beta))/(1-beta*n)**2) +
                                (nu * (un[i, j+1] - 2*un[i, j] + un[i, j-1])/(2*dy)) *
                                (1 + (gamma**2) *
                                (((un[i, j+1] - un[i, j])/dy)**2))**((n1-1)/2) +
                                nu*(n1-1) * (gamma**2) * ((un[i, j+1] - 2*un[i, j] + un[i, j-1])/(2*dy)) * 
                                (((un[i, j+1] - un[i, j])/dy)**2) *
                                (1 + (gamma**2)* 
                                ((un[i, j+1] - un[i, j])/dy)**2)**((n1-3)/2) +
                                mu0*M/rho * (M**2*(-4*h*((4*h**2*(w - i)**2 + Hcalc(M, i, j, h, w) - 
                                ((un[i, j] * (un[i+1, j] - un[i, j])/dx) +
                                 (vn[i, j] * (un[i, j+1] - un[i, j])/dy))) + un[i, j]))))

                #Temperature
                T[i, j] = dt * ((1/(rho*c)) * (kinf * ((Tn[i, j+1] - 2*Tn[i, j] + Tn[i, j-1])/(2*dy)) + 
                               (kinf*eps/delT)* Tn[i ,j] * ((Tn[i, j+1] - Tn[i, j])/dy) - 
                                ((kinf * eps)/delT)*Tinf * ((Tn[i, j+1] - Tn[i, j])/dy)) + 
                                tau * (DB * ((Cn[i, j+1] - Cn[i, j])/dy) * ((Tn[i, j+1] - Tn[i, j])/dy) + 
                                (DT/Tinf) * ((Tn[i, j+1] - Tn[i, j])/dy)**2) + 
                                (Q0/(rho*c)) * (Tn[i, j] - Tinf) - 
                                (un[i, j] * ((Tn[i+1, j] - Tn[i, j])/dx) +
                                 vn[i, j] * ((Tn[i, j+1] - Tn[i, j])/dy))) + Tn[i, j]

                #Concentration
                C[i, j] = dt * (DB * ((Cn[i, j+1] - 2*Cn[i, j] + Cn[i, j-1])/(2*dy)) + 
                                (DT/Tinf) * ((Tn[i, j+1] - 2*Tn[i, j] + Tn[i, j-1])/2*dy) -
                                (un[i, j] * ((Cn[i+1, j] - Cn[i, j])/dx) + 
                                 vn[i, j] * ((Cn[i, j+1] - Cn[i, j])/dy))) + Cn[i, j]
                    

                
                
                udiff = (np.sum(u) - np.sum(un))/np.sum(u)
                Tdiff = (np.sum(T) - np.sum(Tn))/np.sum(T)
                Cdiff = (np.sum(C) - np.sum(Cn))/np.sum(C)
                stepcount += 1